In [ ]:
# TRANING CODE

In [2]:

from tqdm import tqdm
import os, time, shutil, gc, glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import numpy as np
from collections import Counter
import albumentations as A
from albumentations.pytorch import ToTensorV2
import json

os.environ['YOLO_VERBOSE'] = 'False'

################################################################################
# CONFIGURATION
################################################################################
DATA_ROOT = Path('DATASET')
MODEL_SAVE_DIR = Path('SAVED_MODELS')
MODEL_SAVE_DIR.mkdir(exist_ok=True, parents=True)

IMAGE_SIZE_DET = 416
IMAGE_SIZE_CLS = 224
BATCH_SIZE = 12
EPOCHS = 100
PATIENCE = 75
NUM_WORKERS = 0

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"{'='*80}")
print(f"OSU SMALL ANIMALS - YOLO-HARVEST TRAINING")

print(f"{'='*80}")
print(f"Device: {DEVICE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Image Size: {IMAGE_SIZE_CLS}x{IMAGE_SIZE_CLS}")
print(f"{'='*80}\n")

import pickle

def load_clean_dataset():
    """Load pre-validated clean dataset"""
    cache_file = MODEL_SAVE_DIR / 'clean_samples.pkl'
    
    if not cache_file.exists():
        print(" Clean dataset not found!")
        print("Please run: python clean_dataset.py")
        raise FileNotFoundError("Run clean_dataset.py first!")
    
    print("Loading clean dataset...")
    with open(cache_file, 'rb') as f:
        data = pickle.load(f)
    
    print(f" Loaded {len(data['samples']):,} validated images")
    print(f"   ({data['corrupted_count']} corrupted images excluded)")
    
    return data['samples'], data['top_class_to_idx'], data['sub_class_to_idx']

################################################################################
# HARVEST COMPONENTS (PAPER-COMPLIANT)
################################################################################

class ShiftedPatchTokenization(nn.Module):
    """ SPT - Paper Section 3."""
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=384):
        super().__init__()
        self.shift_convs = nn.ModuleList([
            nn.Conv2d(in_chans, embed_dim // 4, kernel_size=patch_size, stride=patch_size)
            for _ in range(4)
        ])
        for conv in self.shift_convs:
            nn.init.kaiming_normal_(conv.weight, mode='fan_out', nonlinearity='relu')
            if conv.bias is not None:
                nn.init.constant_(conv.bias, 0)

    def forward(self, x):
        B, C, H, W = x.shape
        shifts = [(0, 0), (1, 0), (0, 1), (1, 1)]
        shifted_features = []

        for i, (sh, sw) in enumerate(shifts):
            if sh > 0 or sw > 0:
                padded = F.pad(x, (sw, 0, sh, 0))[:, :, :H, :W]
            else:
                padded = x
            feature = self.shift_convs[i](padded)
            shifted_features.append(feature)

        return torch.cat(shifted_features, dim=1)


class LocalInformationEnhancer(nn.Module):
    """LIFE - Paper Section 3.2"""
    def __init__(self, embed_dim=384):
        super().__init__()
        self.dwconv = nn.Conv2d(embed_dim, embed_dim, 3, 1, 1, groups=embed_dim)
        self.bn1 = nn.BatchNorm2d(embed_dim)
        self.pwconv = nn.Conv2d(embed_dim, embed_dim, 1, 1, 0)
        self.bn2 = nn.BatchNorm2d(embed_dim)

    def forward(self, x):
        x = F.gelu(self.bn1(self.dwconv(x)))
        x = F.gelu(self.bn2(self.pwconv(x)))
        return x


class LocalityEnhancedAttention(nn.Module):
    """ LEA - Paper Section 3.3"""
    def __init__(self, dim, num_heads=6, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout)

        self.local_mask = nn.Parameter(torch.randn(num_heads, 1, 1) * 0.02)
        self.rescale = nn.Parameter(torch.ones(1))

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads)
        q, k, v = qkv[:, :, 0], qkv[:, :, 1], qkv[:, :, 2]
        q, k, v = [t.permute(0, 2, 1, 3) for t in (q, k, v)]

        attn = (q @ k.transpose(-2, -1)) * self.scale

        identity = torch.eye(N, device=x.device).unsqueeze(0).unsqueeze(0)
        attn = attn + self.local_mask * identity
        attn = attn * self.rescale

        attn = F.softmax(attn, dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class CrossLevelAttention(nn.Module):
    """Cross-Level Attention for hierarchical consistency"""
    def __init__(self, dim, num_heads=6, dropout=0.1):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(
            dim, num_heads, dropout=dropout, batch_first=True
        )
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_current, x_previous):
        attn_output, _ = self.multihead_attn(
            self.norm(x_current), 
            self.norm(x_previous), 
            self.norm(x_previous)
        )
        return x_current + self.dropout(attn_output)


class HARVESTBlock(nn.Module):
    """ Standard Transformer Block with LEA"""
    def __init__(self, dim, num_heads, mlp_ratio=3.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = LocalityEnhancedAttention(dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(dim)

        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


class HierarchicalHARVESTClassifier(nn.Module):
    
    def __init__(self, input_size=224, embed_dim=384, depth=8, num_heads=6,
                 num_top_classes=1, num_sub_classes=1):
        super().__init__()
        self.input_size = input_size
        self.embed_dim = embed_dim

        self.spt = ShiftedPatchTokenization(input_size, 16, 3, embed_dim)
        self.life = LocalInformationEnhancer(embed_dim)

        num_patches = (input_size // 16) ** 2
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

        self.blocks = nn.ModuleList([
            HARVESTBlock(embed_dim, num_heads, dropout=0.1)
            for _ in range(depth)
        ])

        self.cross_level_attn = CrossLevelAttention(embed_dim, num_heads)
        self.cross_level_position = depth // 2

        self.norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(0.15)

        self.top_class_head = nn.Linear(embed_dim, num_top_classes)
        self.sub_class_head = nn.Linear(
            embed_dim + num_top_classes,
            num_sub_classes
        )

    def forward(self, x):
        x = self.spt(x)
        x = self.life(x)

        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = x + self.pos_embed

        x_mid = None
        for i, block in enumerate(self.blocks):
            x = block(x)
            
            if i == self.cross_level_position - 1:
                x_mid = x.clone()
            elif i == self.cross_level_position and x_mid is not None:
                x = self.cross_level_attn(x, x_mid)

        x = self.norm(x)
        x_pooled = x.mean(dim=1)
        x = self.dropout(x_pooled)

        top_logits = self.top_class_head(x)
        x_sub = torch.cat([x, top_logits], dim=-1)
        sub_logits = self.sub_class_head(x_sub)

        return top_logits, sub_logits

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


################################################################################
# DATASET
################################################################################

class OSUHierarchicalDataset(Dataset):
    def __init__(self, root_dir, mode='train', img_size=IMAGE_SIZE_CLS):
        self.root_dir = Path(root_dir)
        self.mode = mode
        self.img_size = img_size

        self.samples = []
        self.top_class_to_idx = {}
        self.sub_class_to_idx = {}

        print(f"\n{'='*80}")
        print(f"Loading {mode.upper()} dataset from: {root_dir}")
        print(f"{'='*80}")

        self._scan_dataset()

        if mode == 'train':
            self.transform = A.Compose([
                A.RandomResizedCrop(size=(img_size, img_size), scale=(0.7, 1.0), p=1.0),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.3),
                A.RandomRotate90(p=0.5),
                A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.7),
                A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.7),
                A.OneOf([
                    A.GaussNoise(var_limit=(10.0, 50.0)),
                    A.GaussianBlur(blur_limit=(3, 7)),
                    A.MotionBlur(blur_limit=7),
                ], p=0.5),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            self.transform = A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])

        print(f" Dataset loaded: {len(self.samples)} samples")
        print(f"   Top classes: {len(self.top_class_to_idx)}")
        print(f"   Sub classes: {len(self.sub_class_to_idx)}")
        print(f"    Corrupted images will be skipped during training")
        print(f"{'='*80}\n")

    def _scan_dataset(self):
        top_class_idx = 0
        sub_class_idx = 0

        for top_class_dir in sorted(self.root_dir.iterdir()):
            if not top_class_dir.is_dir():
                continue

            top_class_name = top_class_dir.name
            if top_class_name not in self.top_class_to_idx:
                self.top_class_to_idx[top_class_name] = top_class_idx
                top_class_idx += 1

            for sub_class_dir in sorted(top_class_dir.iterdir()):
                if not sub_class_dir.is_dir():
                    continue

                sub_class_name = sub_class_dir.name
                if sub_class_name not in self.sub_class_to_idx:
                    self.sub_class_to_idx[sub_class_name] = sub_class_idx
                    sub_class_idx += 1

                image_extensions = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
                for img_path in sub_class_dir.iterdir():
                    if img_path.suffix in image_extensions:
                        self.samples.append({
                            'path': str(img_path),
                            'top_class': self.top_class_to_idx[top_class_name],
                            'sub_class': self.sub_class_to_idx[sub_class_name],
                            'top_class_name': top_class_name,
                            'sub_class_name': sub_class_name
                        })

        print(f"   Found {len(self.samples)} images")
        print(f"   Top-level classes: {list(self.top_class_to_idx.keys())}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        
        from PIL import ImageFile
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        
        sample = self.samples[idx % len(self.samples)]
        
        try:
            # Load image
            img = Image.open(sample['path'])
            img.load()  # Force immediate load
            img_rgb = img.convert('RGB')
            img_array = np.array(img_rgb)
            img.close()
            
            # Transform
            transformed = self.transform(image=img_array)
            img_tensor = transformed['image']
            
            return img_tensor, sample['top_class'], sample['sub_class'], sample['path']
        
        except:
            # Return dummy on ANY error
            return (
                torch.zeros(3, self.img_size, self.img_size),
                -1,  # Will be filtered out
                -1,
                "ERROR"
            )





def augment_to_median(dataset):
    print(f"\n{'='*80}")
    print("BALANCING STRATEGY: AUGMENT TO MEDIAN")
    print(f"{'='*80}")

    sub_class_counts = Counter()
    for sample in dataset.samples:
        sub_class_counts[sample['sub_class_name']] += 1

    counts_list = list(sub_class_counts.values())
    median_count = int(np.median(counts_list))

    print(f"\nSub-class distribution (BEFORE balancing):")
    print(f"  Min: {min(counts_list)} | Max: {max(counts_list)} | Median: {median_count}")

    balanced_samples = list(dataset.samples)

    import random
    random.seed(42)

    for sub_class_name, orig_count in sub_class_counts.items():
        if orig_count < median_count:
            class_samples = [s for s in dataset.samples if s['sub_class_name'] == sub_class_name]
            needed = median_count - orig_count

            print(f"  {sub_class_name}: {orig_count} → {median_count} (+{needed})")

            for i in range(needed):
                source_sample = random.choice(class_samples)
                balanced_samples.append(source_sample.copy())

    print(f"\n✅ Balanced: {len(dataset.samples)} → {len(balanced_samples)}")
    print(f"{'='*80}\n")

    return balanced_samples


################################################################################
# YOLO-HARVEST MODEL 
################################################################################

try:
    from ultralytics import YOLO
except ImportError:
    os.system('pip install --quiet ultralytics')
    from ultralytics import YOLO


class YOLO_HARVEST_Production(nn.Module):
    
    def __init__(self, num_top_classes, num_sub_classes, yolo_model='yolov8n.pt'):
        super().__init__()

        
        print(f"\n Loading YOLO detector: {yolo_model}")
        yolo_detector = YOLO(yolo_model)
        
        # Store YOLO in a way that doesn't interfere with nn.Module
        object.__setattr__(self, '_yolo_detector', yolo_detector)
        print(" YOLO loaded (for production detection)")

        self.harvest = HierarchicalHARVESTClassifier(
            input_size=IMAGE_SIZE_CLS,
            embed_dim=384,
            depth=8,
            num_heads=6,
            num_top_classes=num_top_classes,
            num_sub_classes=num_sub_classes
        )

        print(f" HARVEST initialized")
        print(f"   Parameters: {self.harvest.count_parameters():,}")
        print(f"   Cross-level attention:  Enabled")
        print(f"   Progressive heads:  Enabled")
        print(f"   Top classes: {num_top_classes}")
        print(f"   Sub classes: {num_sub_classes}\n")

    def forward(self, images_tensor, image_paths=None, use_yolo=False):
        """
        Training mode: Direct classification (faster)
        Production mode: YOLO detection + classification
        """
        if use_yolo and image_paths is not None:
            
            yolo_detector = object.__getattribute__(self, '_yolo_detector')
            batch_crops = []
            for img_path in image_paths:
                results = yolo_detector(img_path, verbose=False)
                # TODO: Crop detected regions and classify
                # For now, use full image
                batch_crops.append(images_tensor)
            
            top_logits, sub_logits = self.harvest(images_tensor)
        else:
            
            top_logits, sub_logits = self.harvest(images_tensor)

        return top_logits, sub_logits


################################################################################
# CHECKPOINT MANAGEMENT
################################################################################

def find_latest_checkpoint(save_dir):
    checkpoint_pattern = str(save_dir / 'best_model_epoch_*.pth')
    checkpoints = glob.glob(checkpoint_pattern)
    
    if not checkpoints:
        return None
    
    epoch_nums = []
    for ckpt in checkpoints:
        try:
            epoch_str = ckpt.split('epoch_')[1].split('_')[0]
            epoch_nums.append((int(epoch_str), ckpt))
        except:
            continue
    
    if not epoch_nums:
        return None
    
    latest_epoch, latest_path = max(epoch_nums, key=lambda x: x[0])
    return latest_path


def load_checkpoint(model, optimizer, scheduler, checkpoint_path):
    print(f"\n{'='*80}")
    print(f" RESUMING FROM CHECKPOINT")
    print(f"{'='*80}")
    print(f"Checkpoint: {checkpoint_path}")
    
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if 'scheduler_state_dict' in checkpoint and scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint.get('epoch', 0)
    best_val_acc = checkpoint.get('val_sub_acc', 0.0)
    
    print(f"Resumed from epoch: {start_epoch}")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    print(f"{'='*80}\n")
    
    return start_epoch, best_val_acc


################################################################################
# TRAINING
################################################################################

class MetricsTracker:
    def __init__(self):
        self.reset()

    def reset(self):
        self.top_correct = 0
        self.sub_correct = 0
        self.total = 0
        self.losses = []

    def update(self, top_preds, sub_preds, top_targets, sub_targets, loss_value):
        if len(top_preds) > 0:
            self.top_correct += (top_preds == top_targets).sum().item()
            self.sub_correct += (sub_preds == sub_targets).sum().item()
            self.total += len(top_targets)
            self.losses.append(float(loss_value))

    def get_top_accuracy(self):
        return self.top_correct / self.total if self.total > 0 else 0.0

    def get_sub_accuracy(self):
        return self.sub_correct / self.total if self.total > 0 else 0.0

    def get_avg_loss(self):
        return np.mean(self.losses) if self.losses else 0.0


def train_harvest(model, train_loader, val_loader, epochs=EPOCHS, resume_checkpoint=None):
    print(f"{'='*80}")
    print("STARTING TRAINING")
    print(f"{'='*80}\n")

    optimizer = optim.AdamW(model.harvest.parameters(), lr=0.001, weight_decay=0.05)
    
    
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-6
    )
    print(f"✅ Using CosineAnnealingWarmRestarts scheduler")
    print(f"   T_0=10 (restart every 10 epochs)")
    print(f"   T_mult=2 (double restart period each time)")
    print(f"   eta_min=1e-6 (minimum LR)")

    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scaler = GradScaler() if DEVICE.type == 'cuda' else None

    start_epoch = 0
    best_val_sub_acc = 0.0
    patience_counter = 0

    if resume_checkpoint:
        start_epoch, best_val_sub_acc = load_checkpoint(
            model, optimizer, scheduler, resume_checkpoint
        )
        
       
        if best_val_sub_acc >= 0.69:  # If stuck around 69%
            for param_group in optimizer.param_groups:
                param_group['lr'] = 0.0008  # Boost LR
            print(f" LR boosted to 0.0008 to escape plateau at {best_val_sub_acc:.4f}")
    for epoch in range(start_epoch, epochs):
        epoch_start = time.time()

        # Training phase
        model.harvest.train()
        train_metrics = MetricsTracker()

        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        for imgs, top_labels, sub_labels, paths in train_pbar:
            imgs = imgs.to(DEVICE, non_blocking=True)
            top_labels = top_labels.to(DEVICE, non_blocking=True)
            sub_labels = sub_labels.to(DEVICE, non_blocking=True)

            valid_mask = (top_labels != -1) & (sub_labels != -1)
            if not valid_mask.any():
                continue

            imgs = imgs[valid_mask]
            top_labels = top_labels[valid_mask]
            sub_labels = sub_labels[valid_mask]

            optimizer.zero_grad()

            try:
                if scaler:
                    with autocast():
                        top_logits, sub_logits = model(imgs)
                        loss_top = criterion(top_logits, top_labels)
                        loss_sub = criterion(sub_logits, sub_labels)
                        loss = 0.3 * loss_top + 0.7 * loss_sub

                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.harvest.parameters(), 1.0)
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    top_logits, sub_logits = model(imgs)
                    loss_top = criterion(top_logits, top_labels)
                    loss_sub = criterion(sub_logits, sub_labels)
                    loss = 0.3 * loss_top + 0.7 * loss_sub

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.harvest.parameters(), 1.0)
                    optimizer.step()

                scheduler.step()

                top_preds = torch.argmax(top_logits, dim=1)
                sub_preds = torch.argmax(sub_logits, dim=1)

                train_metrics.update(top_preds, sub_preds, top_labels, sub_labels, loss.item())

                train_pbar.set_postfix({
                    'Loss': f'{train_metrics.get_avg_loss():.4f}',
                    'Top': f'{train_metrics.get_top_accuracy():.4f}',
                    'Sub': f'{train_metrics.get_sub_accuracy():.4f}'
                })

            except Exception as e:
                print(f"Training error: {e}")
                continue

        # Validation phase
        model.harvest.eval()
        val_metrics = MetricsTracker()

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]  ')
            for imgs, top_labels, sub_labels, paths in val_pbar:
                imgs = imgs.to(DEVICE, non_blocking=True)
                top_labels = top_labels.to(DEVICE, non_blocking=True)
                sub_labels = sub_labels.to(DEVICE, non_blocking=True)

                valid_mask = (top_labels != -1) & (sub_labels != -1)
                if not valid_mask.any():
                    continue

                imgs = imgs[valid_mask]
                top_labels = top_labels[valid_mask]
                sub_labels = sub_labels[valid_mask]

                try:
                    top_logits, sub_logits = model(imgs)
                    loss_top = criterion(top_logits, top_labels)
                    loss_sub = criterion(sub_logits, sub_labels)
                    loss = 0.3 * loss_top + 0.7 * loss_sub

                    top_preds = torch.argmax(top_logits, dim=1)
                    sub_preds = torch.argmax(sub_logits, dim=1)

                    val_metrics.update(top_preds, sub_preds, top_labels, sub_labels, loss.item())

                    val_pbar.set_postfix({
                        'Loss': f'{val_metrics.get_avg_loss():.4f}',
                        'Top': f'{val_metrics.get_top_accuracy():.4f}',
                        'Sub': f'{val_metrics.get_sub_accuracy():.4f}'
                    })

                except Exception as e:
                    continue

        # Epoch summary
        epoch_time = time.time() - epoch_start
        val_sub_acc = val_metrics.get_sub_accuracy()

        print(f"\n{'='*80}")
        print(f"EPOCH {epoch+1}/{epochs} ({epoch_time:.1f}s)")
        print(f"{'='*80}")
        print(f"TRAIN - Loss: {train_metrics.get_avg_loss():.4f} | "
              f"Top: {train_metrics.get_top_accuracy():.4f} | Sub: {train_metrics.get_sub_accuracy():.4f}")
        print(f"VAL   - Loss: {val_metrics.get_avg_loss():.4f} | "
              f"Top: {val_metrics.get_top_accuracy():.4f} | Sub: {val_sub_acc:.4f}")
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")

        
       

        if val_sub_acc > best_val_sub_acc:
            best_val_sub_acc = val_sub_acc
            patience_counter = 0

            model_path = MODEL_SAVE_DIR / f'best_model_epoch_{epoch+1}_acc_{val_sub_acc:.4f}.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),  #  Save scheduler state
                'val_sub_acc': val_sub_acc,
                'val_top_acc': val_metrics.get_top_accuracy()
            }, model_path)

            print(f" BEST MODEL SAVED: {model_path.name}")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{PATIENCE})")

        print(f"{'='*80}\n")

        if patience_counter >= PATIENCE:
            print(f"Early stopping triggered")
            break

        if DEVICE.type == 'cuda':
            torch.cuda.empty_cache()
        gc.collect()

    print(f"\n{'='*80}")
    print("TRAINING COMPLETED")
    print(f"{'='*80}")
    print(f"Best validation accuracy: {best_val_sub_acc:.4f}")
    print(f"{'='*80}\n")

    return model



################################################################################
# MAIN
################################################################################
if __name__ == '__main__':
    print("\n OSU SMALL ANIMALS - YOLO-HARVEST  TRAINING")
  

    try:
        latest_checkpoint = find_latest_checkpoint(MODEL_SAVE_DIR)
        
        if latest_checkpoint:
            print(f"\n Found checkpoint: {latest_checkpoint}")
            resume = input("Resume training? (y/n): ").strip().lower()
            if resume != 'y':
                latest_checkpoint = None
        else:
            print("\n No checkpoints found. Starting from scratch.")

        print("\nLoading dataset...")
        all_samples_dataset = OSUHierarchicalDataset(DATA_ROOT, mode='train')
        from sklearn.model_selection import train_test_split
        train_samples, val_samples = train_test_split(
            all_samples_dataset.samples,
            test_size=0.2,
            random_state=42
        )

        print(f"\nDataset split:")
        print(f"  Training: {len(train_samples)}")
        print(f"  Validation: {len(val_samples)}")

        train_dataset = OSUHierarchicalDataset(DATA_ROOT, mode='train')
        train_dataset.samples = train_samples
        train_dataset.top_class_to_idx = all_samples_dataset.top_class_to_idx
        train_dataset.sub_class_to_idx = all_samples_dataset.sub_class_to_idx

        balanced_samples = augment_to_median(train_dataset)
        train_dataset.samples = balanced_samples

        val_dataset = OSUHierarchicalDataset(DATA_ROOT, mode='val')
        val_dataset.samples = val_samples
        val_dataset.top_class_to_idx = all_samples_dataset.top_class_to_idx
        val_dataset.sub_class_to_idx = all_samples_dataset.sub_class_to_idx

        mappings = {
            'top_class_to_idx': all_samples_dataset.top_class_to_idx,
            'sub_class_to_idx': all_samples_dataset.sub_class_to_idx
        }
        with open(MODEL_SAVE_DIR / 'class_mappings.json', 'w') as f:
            json.dump(mappings, f, indent=2)
        print(f" Class mappings saved\n")

        train_loader = DataLoader(
            train_dataset, batch_size=BATCH_SIZE, shuffle=True,
            num_workers=NUM_WORKERS, pin_memory=False, drop_last=True
        )
        val_loader = DataLoader(
            val_dataset, batch_size=BATCH_SIZE, shuffle=False,
            num_workers=NUM_WORKERS, pin_memory=False
        )

        num_top_classes = len(all_samples_dataset.top_class_to_idx)
        num_sub_classes = len(all_samples_dataset.sub_class_to_idx)

        model = YOLO_HARVEST_Production(num_top_classes, num_sub_classes).to(DEVICE)

        model = train_harvest(
            model, train_loader, val_loader, 
            epochs=EPOCHS, 
            resume_checkpoint=latest_checkpoint
        )

        print("\n TRAINING COMPLETED SUCCESSFULLY!")

    except Exception as e:
        print(f"\n Error: {e}")
        import traceback
        traceback.print_exc()
        raise




ModuleNotFoundError: No module named 'torch'

In [ ]:
# TESTING CODE

In [ ]:


import os
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    roc_curve, auc, precision_recall_curve,
    top_k_accuracy_score
)
from sklearn.preprocessing import label_binarize
import cv2
from PIL import Image




################################################################################
# CONFIGURATION
################################################################################

DATA_ROOT = Path('DATASET')
MODEL_PATH = Path('YOUR_TARINED_MODEL')
CLASS_MAPPINGS = Path('class_mappings.json')
EVAL_DIR = Path('EVALUATION_RESULTS')
EVAL_DIR.mkdir(exist_ok=True, parents=True)

IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 0
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"\n{'='*80}")
print(f"YOLO-HARVEST MODEL EVALUATION")
print(f"{'='*80}")
print(f"Device: {DEVICE}")
print(f"Model: {MODEL_PATH.name}")
print(f"Results: {EVAL_DIR}")
print(f"{'='*80}\n")

################################################################################
# LOAD MODEL AND DATA
################################################################################

def load_model_and_data():
    """Load trained model and validation dataset"""
    
    print("Loading class mappings...")
    with open(CLASS_MAPPINGS, 'r') as f:
        mappings = json.load(f)
    
    top_class_to_idx = mappings['top_class_to_idx']
    sub_class_to_idx = mappings['sub_class_to_idx']
    
    # Reverse mappings for display
    idx_to_top_class = {v: k for k, v in top_class_to_idx.items()}
    idx_to_sub_class = {v: k for k, v in sub_class_to_idx.items()}
    
    num_top_classes = len(top_class_to_idx)
    num_sub_classes = len(sub_class_to_idx)
    
    print(f"Classes: {num_top_classes} top-level, {num_sub_classes} sub-level")
    
    # Load model
    print("Loading model...")
    model = YOLO_HARVEST_Production(
        num_top_classes=num_top_classes,
        num_sub_classes=num_sub_classes
    ).to(DEVICE)
    
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f" Model loaded (Epoch {checkpoint['epoch']}, Val Acc: {checkpoint['val_sub_acc']:.4f})")
    
    # Load validation dataset
    print("Loading validation dataset...")
    from sklearn.model_selection import train_test_split
    
    base_dataset = OSUHierarchicalDataset(DATA_ROOT, mode='train')
    _, val_samples = train_test_split(
        base_dataset.samples, test_size=0.2, random_state=42
    )
    
    val_dataset = OSUHierarchicalDataset(DATA_ROOT, mode='val')
    val_dataset.samples = val_samples
    val_dataset.top_class_to_idx = top_class_to_idx
    val_dataset.sub_class_to_idx = sub_class_to_idx
    
    val_loader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, 
        shuffle=False, num_workers=NUM_WORKERS, pin_memory=False
    )
    
    print(f" Validation set: {len(val_dataset)} samples\n")
    
    return model, val_loader, idx_to_top_class, idx_to_sub_class

################################################################################
# PREDICTION AND METRICS COLLECTION
################################################################################

def collect_predictions(model, val_loader):
    """Collect all predictions and ground truth labels"""
    
    print("Collecting predictions...")
    
    all_top_preds = []
    all_top_labels = []
    all_top_probs = []
    
    all_sub_preds = []
    all_sub_labels = []
    all_sub_probs = []
    
    all_paths = []
    
    with torch.no_grad():
        for imgs, top_labels, sub_labels, paths in tqdm(val_loader, desc="Evaluating"):
            imgs = imgs.to(DEVICE, non_blocking=True)
            top_labels = top_labels.to(DEVICE, non_blocking=True)
            sub_labels = sub_labels.to(DEVICE, non_blocking=True)
            
            # Filter valid samples
            valid_mask = (top_labels != -1) & (sub_labels != -1)
            if not valid_mask.any():
                continue
            
            imgs = imgs[valid_mask]
            top_labels = top_labels[valid_mask]
            sub_labels = sub_labels[valid_mask]
            paths = [p for i, p in enumerate(paths) if valid_mask[i]]
            
            # Forward pass
            top_logits, sub_logits = model(imgs)
            
            # Top-level predictions
            top_probs = F.softmax(top_logits, dim=1)
            top_preds = torch.argmax(top_probs, dim=1)
            
            # Sub-level predictions
            sub_probs = F.softmax(sub_logits, dim=1)
            sub_preds = torch.argmax(sub_probs, dim=1)
            
            # Collect
            all_top_preds.extend(top_preds.cpu().numpy())
            all_top_labels.extend(top_labels.cpu().numpy())
            all_top_probs.extend(top_probs.cpu().numpy())
            
            all_sub_preds.extend(sub_preds.cpu().numpy())
            all_sub_labels.extend(sub_labels.cpu().numpy())
            all_sub_probs.extend(sub_probs.cpu().numpy())
            
            all_paths.extend(paths)
    
    results = {
        'top_preds': np.array(all_top_preds),
        'top_labels': np.array(all_top_labels),
        'top_probs': np.array(all_top_probs),
        'sub_preds': np.array(all_sub_preds),
        'sub_labels': np.array(all_sub_labels),
        'sub_probs': np.array(all_sub_probs),
        'paths': all_paths
    }
    
    print(f" Collected {len(all_sub_preds)} predictions\n")
    
    return results

################################################################################
# VISUALIZATION FUNCTIONS
################################################################################

def plot_confusion_matrix(y_true, y_pred, class_names, title, save_path):
    """Plot confusion matrix"""
    
    # Specify all labels to ensure matrix includes all classes
    all_labels = list(range(len(class_names)))
    cm = confusion_matrix(y_true, y_pred, labels=all_labels)
    
    # Normalize
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Handle division by zero for classes with no samples
    cm_norm = np.nan_to_num(cm_norm, nan=0.0)
    
    plt.figure(figsize=(20, 16))
    sns.heatmap(cm_norm, annot=False, fmt='.2f', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Normalized Count'})
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('True Label', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.xticks(rotation=90, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f" Saved: {save_path.name}")



def plot_per_class_accuracy(y_true, y_pred, class_names, title, save_path):
    """Plot per-class accuracy bar chart"""
    
    per_class_acc = []
    for i in range(len(class_names)):
        mask = y_true == i
        if mask.sum() > 0:
            acc = (y_pred[mask] == i).sum() / mask.sum()
            per_class_acc.append(acc)
        else:
            per_class_acc.append(0.0)
    
    # Sort by accuracy
    sorted_indices = np.argsort(per_class_acc)
    sorted_names = [class_names[i] for i in sorted_indices]
    sorted_accs = [per_class_acc[i] for i in sorted_indices]
    
    plt.figure(figsize=(14, 12))
    colors = ['red' if acc < 0.7 else 'orange' if acc < 0.85 else 'green' 
              for acc in sorted_accs]
    plt.barh(sorted_names, sorted_accs, color=colors, alpha=0.7)
    plt.xlabel('Accuracy', fontsize=14)
    plt.title(title, fontsize=16, fontweight='bold')
    plt.axvline(x=np.mean(sorted_accs), color='blue', linestyle='--', 
                linewidth=2, label=f'Mean: {np.mean(sorted_accs):.4f}')
    plt.legend(fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Saved: {save_path.name}")
    
    return per_class_acc


def plot_roc_curves(y_true, y_probs, class_names, title, save_path, max_classes=10):
    """Plot ROC curves for top classes"""
    
    # Binarize labels with all possible classes
    all_labels = list(range(len(class_names)))
    y_true_bin = label_binarize(y_true, classes=all_labels)
    
    # Calculate ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(len(class_names)):
        # Skip classes with no samples
        if y_true_bin[:, i].sum() == 0:
            continue
            
        try:
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_probs[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        except ValueError:
            # Handle edge cases
            continue
    
    # Plot top N classes by sample count
    class_counts = np.bincount(y_true, minlength=len(class_names))
    top_classes = np.argsort(class_counts)[::-1][:max_classes]
    
    # Filter to only classes we have ROC data for
    top_classes = [i for i in top_classes if i in fpr]
    
    plt.figure(figsize=(12, 8))
    for i in top_classes:
        plt.plot(fpr[i], tpr[i], linewidth=2,
                label=f'{class_names[i][:20]} (AUC={roc_auc[i]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title(title, fontsize=16, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f" Saved: {save_path.name}")



def plot_precision_recall_curves(y_true, y_probs, class_names, title, save_path, max_classes=10):
    """Plot Precision-Recall curves"""
    
    all_labels = list(range(len(class_names)))
    y_true_bin = label_binarize(y_true, classes=all_labels)
    
    precision = dict()
    recall = dict()
    pr_auc = dict()
    
    for i in range(len(class_names)):
        # Skip classes with no samples
        if y_true_bin[:, i].sum() == 0:
            continue
            
        try:
            precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_probs[:, i])
            pr_auc[i] = auc(recall[i], precision[i])
        except ValueError:
            continue
    
    class_counts = np.bincount(y_true, minlength=len(class_names))
    top_classes = np.argsort(class_counts)[::-1][:max_classes]
    
    # Filter to only classes we have PR data for
    top_classes = [i for i in top_classes if i in precision]
    
    plt.figure(figsize=(12, 8))
    for i in top_classes:
        plt.plot(recall[i], precision[i], linewidth=2,
                label=f'{class_names[i][:20]} (AUC={pr_auc[i]:.3f})')
    
    plt.xlabel('Recall', fontsize=14)
    plt.ylabel('Precision', fontsize=14)
    plt.title(title, fontsize=16, fontweight='bold')
    plt.legend(loc="lower left", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f" Saved: {save_path.name}")


def analyze_errors(results, idx_to_top, idx_to_sub, save_path):
    """Analyze common misclassifications"""
    
    sub_labels = results['sub_labels']
    sub_preds = results['sub_preds']
    
    # Find misclassified samples
    wrong_mask = sub_labels != sub_preds
    wrong_true = sub_labels[wrong_mask]
    wrong_pred = sub_preds[wrong_mask]
    
    # Count misclassification pairs
    error_pairs = {}
    for true, pred in zip(wrong_true, wrong_pred):
        pair = (true, pred)
        error_pairs[pair] = error_pairs.get(pair, 0) + 1
    
    # Sort by frequency
    sorted_errors = sorted(error_pairs.items(), key=lambda x: x[1], reverse=True)
    
    # Top 20 errors
    top_errors = sorted_errors[:20]
    
    df = pd.DataFrame([
        {
            'True Class': idx_to_sub[pair[0]],
            'Predicted Class': idx_to_sub[pair[1]],
            'Count': count
        }
        for pair, count in top_errors
    ])
    
    df.to_csv(save_path, index=False)
    print(f" Saved: {save_path.name}")
    
    return df


def plot_top_k_accuracy(y_true, y_probs, k_values=[1, 3, 5, 10], save_path=None):
    """Plot Top-K accuracy"""
    
    # Get total number of classes from y_probs shape
    num_classes = y_probs.shape[1]
    all_labels = list(range(num_classes))
    
    top_k_accs = []
    for k in k_values:
        # Pass labels parameter to handle missing classes
        acc = top_k_accuracy_score(y_true, y_probs, k=k, labels=all_labels)
        top_k_accs.append(acc)
        print(f"  Top-{k} Accuracy: {acc:.4f}")
    
    if save_path:
        plt.figure(figsize=(10, 6))
        plt.plot(k_values, top_k_accs, marker='o', linewidth=3, markersize=10)
        plt.xlabel('K (Top-K)', fontsize=14)
        plt.ylabel('Accuracy', fontsize=14)
        plt.title('Top-K Classification Accuracy', fontsize=16, fontweight='bold')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f" Saved: {save_path.name}")



################################################################################
# MAIN EVALUATION
################################################################################

def run_evaluation():
    """Run complete evaluation"""
    
    # Load model and data
    model, val_loader, idx_to_top, idx_to_sub = load_model_and_data()
    
    # Collect predictions
    results = collect_predictions(model, val_loader)
    
    # Extract data
    top_labels = results['top_labels']
    top_preds = results['top_preds']
    top_probs = results['top_probs']
    
    sub_labels = results['sub_labels']
    sub_preds = results['sub_preds']
    sub_probs = results['sub_probs']
    
    # Class names
    top_class_names = [idx_to_top[i] for i in range(len(idx_to_top))]
    sub_class_names = [idx_to_sub[i] for i in range(len(idx_to_sub))]
    
    print(f"\n{'='*80}")
    print("EVALUATION METRICS")
    print(f"{'='*80}\n")
    
    # Overall accuracy
    top_acc = (top_preds == top_labels).mean()
    sub_acc = (sub_preds == sub_labels).mean()
    
    print(f"Top-Level Accuracy:  {top_acc:.4f} ({top_acc*100:.2f}%)")
    print(f"Sub-Level Accuracy:  {sub_acc:.4f} ({sub_acc*100:.2f}%)")
    
    # Top-K accuracy
    print("\nTop-K Sub-Class Accuracy:")
    plot_top_k_accuracy(sub_labels, sub_probs, k_values=[1, 3, 5, 10], 
                       save_path=EVAL_DIR / 'top_k_accuracy.png')
    
    # Confusion matrices
    print(f"\n{'='*80}")
    print("GENERATING VISUALIZATIONS")
    print(f"{'='*80}\n")
    
    plot_confusion_matrix(
        top_labels, top_preds, top_class_names,
        'Top-Level Classification Confusion Matrix',
        EVAL_DIR / 'confusion_matrix_top.png'
    )
    
    plot_confusion_matrix(
        sub_labels, sub_preds, sub_class_names,
        'Sub-Level Classification Confusion Matrix (45 Classes)',
        EVAL_DIR / 'confusion_matrix_sub.png'
    )
    
    # Per-class accuracy
    per_class_acc_top = plot_per_class_accuracy(
        top_labels, top_preds, top_class_names,
        'Per-Class Accuracy - Top Level (7 Classes)',
        EVAL_DIR / 'per_class_accuracy_top.png'
    )
    
    per_class_acc_sub = plot_per_class_accuracy(
        sub_labels, sub_preds, sub_class_names,
        'Per-Class Accuracy - Sub Level (45 Classes)',
        EVAL_DIR / 'per_class_accuracy_sub.png'
    )
    
    # ROC curves
    plot_roc_curves(
        sub_labels, sub_probs, sub_class_names,
        'ROC Curves - Top 10 Sub-Classes by Sample Count',
        EVAL_DIR / 'roc_curves_sub.png',
        max_classes=10
    )
    
    # Precision-Recall curves
    plot_precision_recall_curves(
        sub_labels, sub_probs, sub_class_names,
        'Precision-Recall Curves - Top 10 Sub-Classes',
        EVAL_DIR / 'precision_recall_sub.png',
        max_classes=10
    )
    
    # Error analysis
    error_df = analyze_errors(
        results, idx_to_top, idx_to_sub,
        EVAL_DIR / 'top_20_errors.csv'
    )
    
    # Classification report
    print("\nGenerating classification reports...")

# Specify all possible labels to handle missing classes
    all_sub_labels = list(range(len(sub_class_names)))

    report_sub = classification_report(
        sub_labels, sub_preds, 
        target_names=sub_class_names,
        labels=all_sub_labels,
        zero_division=0,  # Handle classes with no support
        output_dict=True
    )

    df_report = pd.DataFrame(report_sub).transpose()
    df_report.to_csv(EVAL_DIR / 'classification_report_sub.csv')
    print(f"✅ Saved: classification_report_sub.csv")
    
    # Save summary
    summary = {
        'top_level_accuracy': float(top_acc),
        'sub_level_accuracy': float(sub_acc),
        'num_samples': len(sub_labels),
        'num_top_classes': len(top_class_names),
        'num_sub_classes': len(sub_class_names),
        'per_class_accuracy_sub': {
            sub_class_names[i]: float(per_class_acc_sub[i]) 
            for i in range(len(sub_class_names))
        }
    }
    
    with open(EVAL_DIR / 'evaluation_summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    print(f" Saved: evaluation_summary.json")
    
    print(f"\n{'='*80}")
    print("EVALUATION COMPLETE!")
    print(f"{'='*80}")
    print(f"Results saved to: {EVAL_DIR}")
    print(f"{'='*80}\n")


################################################################################
# MAIN EXECUTION
################################################################################

if __name__ == '__main__':
    try:
        run_evaluation()
        print("\n🎉 Evaluation completed successfully!\n")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
